In [1]:
import pandas as pd

In [2]:
## Code ran on EC2 instance 

In [3]:
gu_sample_content=pd.read_csv('gu_resampled_by_section_id_content.csv',index_col=0, converters={'paragraphs': pd.eval})

In [4]:
gu_sample_content.shape

(54892, 5)

In [5]:
# Explode individual text paragraphs across rows to reorder and resample
gu_sample_content=gu_sample_content[['url','paragraphs','doc_index']].explode(['paragraphs'])

In [6]:
gu_sample_content.dropna(inplace=True)

In [7]:
gu_sample_content.iloc[0]['paragraphs']

'Jurors who sat on the Zachary Rolfe murder trial in the supreme court of the Northern Territory might be feeling cheated.'

In [8]:
gu_sample_content.shape

(917299, 3)

In [9]:
gu_sample_content

,url,paragraphs,doc_index
0,www.theguardian.com/law/commentisfree/2022/mar...,Jurors who sat on the Zachary Rolfe murder tri...,0
0,www.theguardian.com/law/commentisfree/2022/mar...,"A week after the not-guilty verdict came down,...",0
0,www.theguardian.com/law/commentisfree/2022/mar...,This was “tendency evidence” that Justice John...,0
0,www.theguardian.com/law/commentisfree/2022/mar...,The suppressed information included statements...,0
0,www.theguardian.com/law/commentisfree/2022/mar...,Judge Greg Borchers found that Rolfe lacked cr...,0
...,...,...,...
183148,www.theguardian.com/global-development/2016/no...,"Last year, the World Health Organisation and U...",25939
183148,www.theguardian.com/global-development/2016/no...,"Africa’s biggest economy, Nigeria, is falling ...",25939
183148,www.theguardian.com/global-development/2016/no...,"The report ranks South Sudan, the world’s newe...",25939
183148,www.theguardian.com/global-development/2016/no...,All UN member states have committed to ensurin...,25939


In [10]:
# TO DO 
# Clean up liveblogs from URL and then again from the list of relevant articles 

In [9]:
# Read in entities present in the KB to sample paragraphs with full name matches
dataset='full'
kb_iteration='2022_10_19'
dataset=f'{dataset}_{kb_iteration}'
entity_data=pd.read_csv(f'../kb_datasets/kb_entities_{dataset}.csv',index_col=0)

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (2,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
aliases=entity_data['name'].str.lower()
#pd.DataFrame(aliases,columns=['name']) .to_csv('gu_resampled_by_section_id_aliases.csv')

In [11]:
gu_entities=pd.read_csv('unique_gu_sample_person_entities.csv',index_col=0)

In [12]:
gu_entities.dropna(inplace=True)

In [13]:
gu_entities.rename(columns={'text':'gu_ents'},inplace=True)

In [14]:
gu_ents_set=set(gu_entities['gu_ents'].str.lower())
common_ents=set(gu_ents_set).intersection(set(aliases))

In [15]:
print('There are {} common entities between kb and gu article sample'.format(len(common_ents)))

There are 31360 common entities between kb and gu article sample


In [16]:
gu_entities=gu_entities[gu_entities['gu_ents'].str.lower().isin(common_ents)]

In [17]:
len(common_ents)

31360

In [18]:
## Deprecated?

In [19]:
from datetime import datetime

In [20]:
aliases=[ent for ent in common_ents if len(ent) > 4]
#aliases=aliases[:10]
gu_sample_content_copy=gu_sample_content.copy()
gu_sample_content_copy=gu_sample_content.reset_index(drop=True)
gu_sample_content_copy['lower_paragraphs']=gu_sample_content_copy['paragraphs'].str.lower()
start_time = datetime.now()
i = 0
first=True
start=datetime.now()
for alias in aliases:
    paragraphs=gu_sample_content_copy[
        gu_sample_content_copy['lower_paragraphs'].str.contains(alias,na=False)
    ].copy()
    paragraphs['alias']=alias
    if first:
        relevant_gu_paragraphs=paragraphs
        first=False
    else:
        relevant_gu_paragraphs=pd.concat([relevant_gu_paragraphs, paragraphs])
    i+=1
    if i%2000==0:
        end_time = datetime.now()
        print('Duration: {}'.format(end_time - start_time))
        print(i)
        print('--')
    gu_sample_content_copy=gu_sample_content_copy.drop(paragraphs.index)
#
end_time=datetime.now()

Duration: 0:27:36.756707
2000
--
Duration: 6:32:51.398745
4000
--
Duration: 6:58:58.482699
6000
--
Duration: 7:24:47.854227
8000
--
Duration: 7:50:14.905105
10000
--
Duration: 8:15:30.500510
12000
--
Duration: 8:40:29.531277
14000
--
Duration: 9:05:20.074737
16000
--
Duration: 9:29:49.626987
18000
--
Duration: 9:54:05.816347
20000
--
Duration: 10:18:03.733875
22000
--
Duration: 10:41:50.320763
24000
--
Duration: 11:05:19.897229
26000
--
Duration: 11:28:39.624376
28000
--
Duration: 11:51:43.672143
30000
--


In [21]:
gu_sample_content_copy.shape

(764661, 4)

In [22]:
print(end_time-start_time)

12:07:00.786260


In [23]:
gu_sample_content_copy.shape

(764661, 4)

In [24]:
gu_sample_content.shape

(917299, 3)

In [25]:
relevant_gu_paragraphs.shape

(152638, 5)

In [26]:
del(gu_sample_content_copy)

In [27]:
relevant_gu_paragraphs.drop_duplicates(inplace=True)

In [28]:
relevant_gu_paragraphs.shape

(152616, 5)

In [29]:
relevant_gu_paragraphs['alias'].value_counts()

donald trump       4255
boris johnson      4219
corby              3945
anita              3129
david cameron      2389
                   ... 
sergey nalobin        1
thomas caldwell       1
melissa sargent       1
rebecca taylor        1
aaron ford            1
Name: alias, Length: 17300, dtype: int64

In [30]:
# Reorder paragraphs and export
relevant_gu_paragraphs.sample(frac=1, random_state=42).to_csv('gu_resampled_by_section_id_full_name_all_aliases_kb_match_paragraphs.csv') 

In [33]:
relevant_gu_paragraphs.shape

(152616, 5)

In [ ]:
### Deprecated?